In [1]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

In [2]:
!ls -l ../data/

total 580596
-rw------- 1 v.bugaevskii v.bugaevskii    904039 Apr 17 11:12 appl.csv
-rw------- 1 v.bugaevskii v.bugaevskii   6577695 Apr 17 11:12 aum.csv
-rw------- 1 v.bugaevskii v.bugaevskii  91636746 Apr 17 11:12 balance.csv
-rw------- 1 v.bugaevskii v.bugaevskii   1110765 Apr 17 11:12 client.csv
-rw------- 1 v.bugaevskii v.bugaevskii   6686165 Apr 17 11:12 com.csv
-rw------- 1 v.bugaevskii v.bugaevskii   7499804 Apr 17 11:12 deals.csv
-rw------- 1 v.bugaevskii v.bugaevskii     18979 Apr 17 11:12 dict_mcc.csv
-rw------- 1 v.bugaevskii v.bugaevskii   2322389 Apr 17 11:12 funnel.csv
-rw------- 1 v.bugaevskii v.bugaevskii  10553763 Apr 17 11:12 payments.csv
-rw------- 1 v.bugaevskii v.bugaevskii 467193115 Apr 17 11:12 trxn.csv


In [3]:
from pathlib import Path

In [4]:
path_df = Path('../data').rglob('*.csv')
path_df = sorted(path_df)

for path in tqdm(path_df):
    df_name = 'df_' + path.name.rsplit('.', 1)[0]
    vars()[df_name] = pd.read_csv(path)
    print(f'{path} saved to {df_name}; {df_name}.shape = {vars()[df_name].shape}')

  0%|          | 0/10 [00:00<?, ?it/s]

../data/appl.csv saved to df_appl; df_appl.shape = (12030, 6)
../data/aum.csv saved to df_aum; df_aum.shape = (117392, 4)
../data/balance.csv saved to df_balance; df_balance.shape = (1194684, 9)
../data/client.csv saved to df_client; df_client.shape = (21498, 8)
../data/com.csv saved to df_com; df_com.shape = (113055, 10)
../data/deals.csv saved to df_deals; df_deals.shape = (109016, 8)
../data/dict_mcc.csv saved to df_dict_mcc; df_dict_mcc.shape = (915, 3)
../data/funnel.csv saved to df_funnel; df_funnel.shape = (21498, 16)
../data/payments.csv saved to df_payments; df_payments.shape = (188068, 4)


/home/v.bugaevskii/.conda/envs/idao2021/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


../data/trxn.csv saved to df_trxn; df_trxn.shape = (3035705, 11)


In [7]:
df_grouped = df_aum.sort_values(by='month_end_dt')\
    .groupby(['client_id', 'product_code'])['balance_rur_amt'].apply(lambda r: r.iloc[-1])
df_grouped = df_grouped.unstack('product_code').fillna(0)
df_grouped['sum'] = df_grouped.sum(axis=1).values
df_grouped = df_grouped.add_prefix('aum_amnt_')
df_grouped.head()

product_code,aum_amnt_Broker,aum_amnt_Current accounts,aum_amnt_ILI,aum_amnt_NSJ,aum_amnt_Time account,aum_amnt_UK,aum_amnt_sum
client_id,,,,,,,
-9220236243053692422,0.0,8552.0,0.0,0.0,0.0,0.0,8552.0
-9220233431709087652,0.0,52.0,0.0,0.0,0.0,0.0,52.0
-9219699286371310531,0.0,3532.0,0.0,0.0,0.0,0.0,3532.0
-9218923580775423458,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9218871523310554579,0.0,40937.0,0.0,0.0,0.0,0.0,40937.0


In [8]:
df_grouped.to_csv('aum_features.csv')